# Language Translation Using Recurring Neural Networks

### Data Loading 

In [3]:
import pandas as pd
import os

In [4]:
for dirname, _, filenames in os.walk('/Capstone/Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Capstone/Data\EN-NL\EN -NL.txt
/Capstone/Data\EN-BG\EN-BG.txt
/Capstone/Data\EN-CS\EN-CS.txt
/Capstone/Data\EN-DA\EN-DA.txt
/Capstone/Data\EN-DE\EN-DE.txt
/Capstone/Data\EN-EL\EN-EL.txt
/Capstone/Data\EN-ES\EN-ES.txt
/Capstone/Data\EN-ET\EN-ET.txt
/Capstone/Data\EN-FI\EN-FI.txt
/Capstone/Data\EN-FR\EN-FR.txt
/Capstone/Data\EN-GA\EN-GA.txt
/Capstone/Data\EN-HR\EN-HR.txt
/Capstone/Data\EN-HU\EN-HU.txt
/Capstone/Data\EN-IT\EN-IT.txt
/Capstone/Data\EN-LT\EN-LT.txt
/Capstone/Data\EN-LV\EN-LV.txt
/Capstone/Data\EN-MT\EN-MT.txt
/Capstone/Data\EN-PL\EN-PL.txt
/Capstone/Data\EN-PT\EN-PT.txt
/Capstone/Data\EN-RO\EN-RO.txt
/Capstone/Data\EN-SK\EN-SK.txt
/Capstone/Data\EN-SL\EN-SL.txt
/Capstone/Data\EN-SV\EN-SV.txt
/Capstone/Data\Sample TMX file (EN-GA)\EN-GA.tmx


In [5]:
CD = "/Capstone/Data/"

In [6]:
SL = 'EN'

In [7]:
TL_DE = 'DE'

In [8]:
TL_ES = 'ES'

In [9]:
EN_DE = pd.read_csv(CD+SL+'-'+TL_DE+'/'+SL+'-'+TL_DE+'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:SL, 1:TL_DE})

In [10]:
EN_ES = pd.read_csv(CD+SL+'-'+TL_ES+'/'+SL+'-'+TL_ES+'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:SL, 1:TL_ES})

In [11]:
EN_DE.head()

,EN,DE
0,Commission Regulation (EC) No 1788/2004,Verordnung (EG) Nr. 1788/2004 der Kommission
1,of 15 October 2004,vom 15. Oktober 2004
2,fixing the minimum selling prices for butter f...,zur Festsetzung der Mindestverkaufspreise für ...
3,"THE COMMISSION OF THE EUROPEAN COMMUNITIES,",DIE KOMMISSION DER EUROPÄISCHEN GEMEINSCHAFTEN —
4,Having regard to the Treaty establishing the E...,gestützt auf den Vertrag zur Gründung der Euro...


In [12]:
EN_ES.head()

,EN,ES
0,Commission Regulation (EC) No 1788/2004,Reglamento (CE) no 1788/2004 de la Comisión
1,of 15 October 2004,de 15 de octubre de 2004
2,fixing the minimum selling prices for butter f...,por el que se fijan los precios mínimos de ven...
3,"THE COMMISSION OF THE EUROPEAN COMMUNITIES,","LA COMISIÓN DE LAS COMUNIDADES EUROPEAS,"
4,Having regard to the Treaty establishing the E...,Visto el Tratado constitutivo de la Comunidad ...


In [13]:
EN_DE.shape

(5693624, 2)

In [14]:
EN_ES.shape

(5696850, 2)

In [15]:
EN_DE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5693624 entries, 0 to 5693623
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   EN      object
 1   DE      object
dtypes: object(2)
memory usage: 86.9+ MB


In [16]:
EN_ES.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5696850 entries, 0 to 5696849
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   EN      object
 1   ES      object
dtypes: object(2)
memory usage: 86.9+ MB


### Data Cleaning

In [18]:
missing_DE = pd.concat([EN_DE.isnull().sum(), 100 * EN_DE.isnull().mean()], axis=1)
missing_DE.columns=['count', '%']
missing_DE.sort_values(by= ['count','%'], ascending=False)

,count,%
EN,134,0.002354
DE,4,0.000070


In [19]:
missing_ES = pd.concat([EN_ES.isnull().sum(), 100 * EN_ES.isnull().mean()], axis=1)
missing_ES.columns=['count', '%']
missing_ES.sort_values(by= ['count','%'], ascending=False)

,count,%
EN,130,0.002282
ES,2,0.000035


In [20]:
EN_DE.dropna(inplace = True)

In [21]:
EN_DE.isna().sum()

EN    0
DE    0
dtype: int64

In [22]:
EN_DE.shape

(5693487, 2)

In [23]:
EN_ES.dropna(inplace = True)

In [24]:
EN_ES.isna().sum()

EN    0
ES    0
dtype: int64

In [ ]:
EN_DE['EN'] = EN_DE['EN'].str.lower().str.split()
EN_DE['DE'] = EN_DE['DE'].str.lower().str.split()

In [ ]:
EN_ES['EN'] = EN_DE['EN'].str.lower().str.split()
EN_ES['ES'] = EN_ES['ES'].str.lower().str.split()

In [ ]:
#EN_complete = pd.merge(EN_DE, EN_ES, on = 'EN')

### Data Visualization 

In [17]:
from collections import Counter

In [2]:

EN_DE['ENCounter'] = EN_DE['DE'].apply(set).apply(len)
#EN_ES['ES'].str.lower.str.split.apply(results_ES.update)

#print('English to German word count: '+ results_DE)
#print(' ')
#print('English to Spanish word count: '+ results_ES)

NameError: name 'EN_DE' is not defined